# Processing multiple interferograms with ISCE

In [1]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-29 19:08:38,313 - matplotlib - DEBUG - matplotlib data path: /home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/matplotlib/mpl-data
2022-09-29 19:08:38,319 - matplotlib - DEBUG - CONFIGDIR=/home/jovyan/.config/matplotlib
2022-09-29 19:08:38,321 - matplotlib - DEBUG - interactive is False
2022-09-29 19:08:38,322 - matplotlib - DEBUG - platform is linux
2022-09-29 19:08:38,323 - matplotlib - DEBUG - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakr

In [2]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)
print(os.environ['PATH'])

/home/jovyan/.local/ARIA-tools/tools/bin:/home/jovyan/.local/ARIA-tools/tools/ARIAtools:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications:/home/jovyan/.local/envs/insar_analysis/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/bin:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications


In [3]:
!which topsApp.py

/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications/topsApp.py


In [4]:
scene_list = ['S1B_IW_SLC__1SDV_20170721T005332_20170721T005359_006579_00B928_CAB5',
              'S1B_IW_SLC__1SDV_20170814T005333_20170814T005400_006929_00C349_C0B0',
              'S1B_IW_SLC__1SDV_20170907T005334_20170907T005401_007279_00CD6E_7850',
             ]

In [5]:
def select_pairs(scene_list, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene
        
    pair_dict = {}
    pair_scenes = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not date1 >= date2 :
                pair_dict[f'{date1}-{date2}'] = [scene_dates[date1], scene_dates[date2]]
                pair_scenes.append(scene_dates[date1])
                pair_scenes.append(scene_dates[date2])
    pair_scenes = [*set(pair_scenes)]
    
    print(f'number of pairs: {len(pair_dict)}')
    
    return pair_dict, pair_scenes

In [6]:
pair_dict, pair_scenes = select_pairs(scene_list, 30)

number of pairs: 2


In [7]:
proc_path = '/home/jovyan/rmnp_landslide/proc'
dem_name = 'usgs_10m.dem.wgs84'

for pair in pair_dict:
    pair_path = f'{proc_path}/{pair}'
    if not os.path.exists(pair_path):
                          os.makedirs(pair_path)
    !cp '{proc_path}/{dem_name}.xml' '{pair_path}/{dem_name}.xml'
    !cp '{proc_path}/{dem_name}' '{pair_path}/{dem_name}'

## Download all SLCs

In [8]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

 ········


In [9]:
# Change to SLC directory
os.chdir(f'{proc_path}/slc')

logging.getLogger("urllib3").setLevel(logging.WARNING)

results = asf.granule_search(pair_scenes)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

## Download orbital files

In [10]:
os.chdir(f'{proc_path}/orbital')

In [11]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

--2022-09-29 19:11:16--  https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5054 (4.9K) [text/plain]
Saving to: ‘fetchOrbit.py’

     0K ....                                                  100% 65.9M=0s

2022-09-29 19:11:17 (65.9 MB/s) - ‘fetchOrbit.py’ saved [5054/5054]



In [12]:
# grab orbital files with fetchOrbit.py
for scene in pair_scenes:
    os.system(f'./fetchOrbit.py -i {scene}')

Reference time:  2017-07-21 00:53:59
Satellite name:  S1B
Reference time:  2017-08-14 00:54:00
Satellite name:  S1B
Reference time:  2017-09-07 00:54:01
Satellite name:  S1B


## Write input files

In [13]:
for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    reference = pair_dict[pair][0]
    secondary = pair_dict[pair][1]
    
    with open('topsApp.xml', 'w') as f:
        f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
    <topsApp>
      <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>
        <component name="reference">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">reference</property>
            <property name="safe">{proc_path}/slc/{reference}.zip</property>
        </component>
        <component name="secondary">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">secondary</property>
            <property name="safe">{proc_path}/slc/{secondary}.zip</property>
        </component>
        <property name="demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="useGPU">True</property>
        <property name="range looks">6</property>
        <property name="azimuth looks">1</property>
        <property name="swaths">[1]</property>
        <property name="region of interest">[40.295, 40.310, -105.69, -105.67]</property>
        <property name="do esd">False</property>
        <property name="do ionospherecorrection">False</property>
        <property name="do unwrap">True</property>
        <property name="unwrapper name">snaphu_mcf</property>
        <property name="do denseoffsets">False</property>
        <property name="geocode demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="geocode list">['merged/topophase.cor', 'merged/filt_topophase.unw', 'merged/filt_topophase.flat']</property>
      </component>
    </topsApp>""")

## TopsApp processing

In [14]:
%%time

for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    !topsApp.py --start=preprocess --end=geocode

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-29 19:11:26,050 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Cannot open PICKLE/startup
Running step preprocess
Input XML files:  ['/vsizip//home/jovyan/rmnp_landslide/proc/slc/S1B_IW_SLC__1SDV_20170721T005332_20170721T005359_006579_00B928_CAB5.zip/S1B_IW_SLC__1SDV_20170721T005332_20170721T005359_006579_00B928_CAB5.SAFE/annotation/s1b-iw1-slc-vv-20170721t005333-20170721t005358-006579-0